In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
fdeconv = np.load('deconv_event_0_0.npz')

In [ ]:
fdeconv.files

In [ ]:
fdeconv['deconv_q'].shape

In [ ]:
def pick_pixel_from_block(pxl_x, pxl_y, data, location, dt):
    pxl_indx = pxl_x - location[0]
    pxl_indy = pxl_y - location[1]
    seq = data[pxl_indx, pxl_indy]
    times = location[2] + np.arange(seq.shape[-1]) * dt

    return times, seq

In [ ]:
print(fdeconv['boffset'], fdeconv['smear_offset'])
dpxl_xy = fdeconv['smear_offset'][:2] - fdeconv['boffset'][:2]

In [ ]:
dpxl_xy

In [ ]:
def filter_pixel_positions(offset, data, qmin, qmax):
    mask = data.sum(axis=-1) > qmin
    mask &= data.sum(axis=-1) < qmax
    pxl_x, pxl_y = offset[0] + np.where(mask)[0], offset[1] + np.where(mask)[1]
    return pxl_x, pxl_y

In [ ]:
filter_pixel_positions(fdeconv['boffset'], fdeconv['deconv_q'], 20, 25)

In [ ]:
times, deconvq = pick_pixel_from_block(139, 71, fdeconv['deconv_q'], fdeconv['boffset'], dt=fdeconv['adc_downsample_factor'])
qtimes, trueq = pick_pixel_from_block(139, 71, fdeconv['smeared_true'], fdeconv['smear_offset'], dt=1)
plt.plot(times, deconvq/fdeconv['adc_downsample_factor'])
plt.plot(qtimes, trueq)

In [ ]:
filter_pixel_positions(fdeconv['smear_offset'], fdeconv['smeared_true'], 1, 5)

In [ ]:
filter_pixel_positions(fdeconv['smear_offset'], fdeconv['smeared_true'], 5, 10)

In [ ]:
def plot_sequences(pxl_x, pxl_y):
    fig, ax = plt.subplots()
    times, deconvq = pick_pixel_from_block(pxl_x, pxl_y, fdeconv['deconv_q'], fdeconv['boffset'], dt=fdeconv['adc_downsample_factor'])
    qtimes, trueq = pick_pixel_from_block(pxl_x, pxl_y, fdeconv['smeared_true'], fdeconv['smear_offset'], dt=1)
    ax.plot(times, deconvq/fdeconv['adc_downsample_factor'], label=f'deconv, sum={np.sum(deconvq):.1f}')
    ax.plot(qtimes, trueq, label=f'smeared true, sum={np.sum(trueq):.1f}')
    ax.legend()
    ax.set_title(f'Pixel indes in TPC: ({pxl_x}, {pxl_y})')
    ax.set_xlabel('ticks [50ns]')
    return fig

In [ ]:
plot_sequences(104, 115);

In [ ]:
plot_sequences(107, 119);

In [ ]:
plot_sequences(139, 71)